[Building A Neural Network](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# GPU is called accelerator
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
# NN derived from nn.Module
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)  # always call the method defined in __init__()
        return logits

In [4]:
# instantiate a momdel object
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)   # generates some random data
logits = model(X)                          # feed into the model, and call forweard() of the model
                                           # never call forward() directly
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


In [6]:
# Let's analyze our model class with some generated random data
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values
# ( the minibatch dimension (at dim=0) is maintained).
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# Linear: y = A * x + b, where x: 1D array of size 784, y: 1D array of size 20
# layer1: 20 by 784 matrix
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(f"hidden size: {hidden1.size()}")

hidden size: torch.Size([3, 20])


In [12]:
# nn.ReLU
print(f"Before ReLU: hidden size: {hidden1.size()}, hidden: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: hidden size: {hidden1.size()}, hidden: {hidden1}")

Before ReLU: hidden size: torch.Size([3, 20]), hidden: tensor([[0.0000, 0.0000, 0.0000, 0.1704, 0.0000, 0.0153, 0.5272, 0.1016, 0.0661,
         1.0873, 0.4141, 0.0000, 0.0000, 0.0000, 0.0000, 0.6854, 0.6781, 0.0000,
         0.3188, 0.0000],
        [0.0761, 0.0000, 0.0390, 0.0000, 0.0000, 0.0708, 0.3741, 0.0879, 0.0000,
         0.6531, 0.3642, 0.0000, 0.0000, 0.0000, 0.0000, 0.5215, 0.0522, 0.0000,
         0.1258, 0.0000],
        [0.0000, 0.0000, 0.0255, 0.0000, 0.0000, 0.4349, 0.2054, 0.0000, 0.0000,
         0.7275, 0.0987, 0.0000, 0.0000, 0.0000, 0.0000, 0.6054, 0.6495, 0.0000,
         0.1143, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: hidden size: torch.Size([3, 20]), hidden: tensor([[0.0000, 0.0000, 0.0000, 0.1704, 0.0000, 0.0153, 0.5272, 0.1016, 0.0661,
         1.0873, 0.4141, 0.0000, 0.0000, 0.0000, 0.0000, 0.6854, 0.6781, 0.0000,
         0.3188, 0.0000],
        [0.0761, 0.0000, 0.0390, 0.0000, 0.0000, 0.0708, 0.3741, 0.0879, 0.0000,
         0.6531, 0.3642, 0.000

In [13]:
# nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [14]:
# nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
# model parameters
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0183, -0.0304, -0.0154,  ..., -0.0049,  0.0189,  0.0348],
        [-0.0021,  0.0325, -0.0170,  ...,  0.0152, -0.0195, -0.0076]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0329, -0.0347], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0319, -0.0338, -0.0267,  ...,  0.0176,  0.0143, -0.0148],
        [-0.0302,  0.0220, -0.0204,  ..., -0.0332,  0.0365,  0.0218]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 